**Acknowledgments**: This code was mainly based on the tutorial by Aakash Kumar Nain, available [here](https://keras.io/examples/vision/image_captioning/), to whom we extend our thanks for their contribution and for making the material publicly available.

## Setup

In [ ]:
pip install tensorflow==2.8.3

In [ ]:
import os, json
import re
import numpy as np
import matplotlib.pyplot as plt
import ast

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.layers import TextVectorization


seed = 111
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/Mestrado/ImageCaptioning/datasets/flickr30k/dataset_flickr30k_all.json', 'r') as captionsJson:
  captions = json.load(captionsJson)
with open('/content/drive/MyDrive/Mestrado/ImageCaptioning/datasets/flickr30k/dataset_flickr30k.json', 'r') as captionsJson:
  info = json.load(captionsJson)

In [ ]:
!tar -xf /content/drive/MyDrive/Mestrado/ImageCaptioning/datasets/flickr30k/flickr30k-images.tar.gz

In [ ]:

# Path to the images
IMAGES_PATH = "flickr30k-images"

# Desired image dimensions
IMAGE_SIZE = (384, 384)

# Fixed length allowed for any sequence
SEQ_LENGTH = 25

# Other training parameters
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE

## Preparing the dataset

In [ ]:

def load_captions_data(captionsfile):
    """Loads captions (text) data and maps them to corresponding images.

    Args:
        filename: Path to the text file containing caption data.

    Returns:
        caption_mapping: Dictionary mapping image names and the corresponding captions
        text_data: List containing all the available captions
    """
    caption_mapping = {}
    text_data = []
    images_to_skip = set()

    for img_name, captions in captionsfile.items():
        img_name = os.path.join(IMAGES_PATH, img_name)
        for caption in captions:
          tokens = caption.strip().split()

          if img_name.endswith("jpg") and img_name not in images_to_skip:
              # We will add a start and an end token to each caption
              text_data.append(caption)

              if img_name in caption_mapping:
                  caption_mapping[img_name].append(caption)
              else:
                  caption_mapping[img_name] = [caption]

    return caption_mapping, text_data

def train_val_split_karpathy(caption_data, info_data):
    """Split the captioning dataset into train and validation sets.

    Args:
        caption_data (dict): Dictionary containing the mapped caption data
        train_size (float): Fraction of all the full dataset to use as training data
        shuffle (bool): Whether to shuffle the dataset before splitting

    Returns:
        Traning and validation datasets as two separated dicts
    """

    # 1. Get the list of all image names
    all_images = list(caption_data.keys())
    all_images = np.array(all_images)
    train_idx, val_idx, test_idx = [], [], []

    for image in info_data['images']:
      if image['split'] == 'train':
        train_idx.append(np.where(all_images==IMAGES_PATH + '/' + image['filename']))
      if image['split'] == 'val':
        val_idx.append(np.where(all_images==IMAGES_PATH + '/' + image['filename']))
      if image['split'] == 'test':
        test_idx.append(np.where(all_images==IMAGES_PATH + '/' + image['filename']))


    training_data = {
        img_name[0][0]: caption_data[img_name[0][0]] for img_name in all_images[train_idx]
    }
    validation_data = {
        img_name[0][0]: caption_data[img_name[0][0]] for img_name in all_images[val_idx]
    }
    test_data = {
        img_name[0][0]: caption_data[img_name[0][0]] for img_name in all_images[test_idx]
    }

    # 4. Return the splits
    return training_data, validation_data, test_data


# Load the dataset
captions_mapping, text_data = load_captions_data(captions)

# Split the dataset into training and validation sets
train_data, valid_data, test_data = train_val_split_karpathy(captions_mapping, info)
print("Number of training samples: ", len(train_data))
print("Number of validation samples: ", len(valid_data))
print("Number of test samples: ", len(test_data))

Number of training samples:  29000
Number of validation samples:  1014
Number of test samples:  1000


In [ ]:
train_data_text = []
for i in train_data.values(): train_data_text.extend(i)

In [ ]:
len(train_data_text), len(text_data)

(145000, 155070)

## Vectorizing the text data

We'll use the `TextVectorization` layer to vectorize the text data,
that is to say, to turn the
original strings into integer sequences where each integer represents the index of
a word in a vocabulary. We will use a custom string standardization scheme
(strip punctuation characters except `<` and `>`) and the default
splitting scheme (split on whitespace).

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~0123456789')
tokenizer.fit_on_texts(train_data_text)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'
VOCAB_SIZE=0
out=0
for word, q in ast.literal_eval(tokenizer.get_config()['word_counts']).items():
  if q > 4: VOCAB_SIZE-=-1
  else: out-=-1
VOCAB_SIZE, out

(8056, 13313)

In [ ]:
d=ast.literal_eval(tokenizer.get_config()['word_counts'])

In [ ]:

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


strip_chars = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
strip_chars = strip_chars.replace("<", "")
strip_chars = strip_chars.replace(">", "")

vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQ_LENGTH,
    standardize=custom_standardization,
    pad_to_max_tokens=True
)
vectorization.adapt(train_data_text)

# Data augmentation for image data
image_augmentation = keras.Sequential(
    [

        layers.RandomTranslation(height_factor=(0.1), width_factor=(0.1),
                                 fill_mode="nearest",),
        layers.RandomZoom(height_factor=(0.1), fill_mode="nearest"),
        layers.RandomRotation(0.1, fill_mode="nearest"),
        layers.RandomHeight(0.2),
        layers.RandomWidth(0.2),
        layers.RandomContrast(0.1),
    ]
)

## Building a `tf.data.Dataset` pipeline for training

We will generate pairs of images and corresponding captions using a `tf.data.Dataset` object.
The pipeline consists of two steps:

1. Read the image from the disk
2. Tokenize all the five captions corresponding to the image

In [ ]:
images, captions = list(train_data.keys()), list(train_data.values())

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(5,), dtype=tf.string, name=None))>

In [ ]:

def decode_and_resize(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img


def process_input(img_path, captions):
    return decode_and_resize(img_path), vectorization(captions)


def make_dataset(images, captions):
    dataset = tf.data.Dataset.from_tensor_slices((images, captions))
    dataset = dataset.shuffle(len(images))
    dataset = dataset.map(process_input, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

    return dataset


# Pass the list of images and the list of corresponding captions
train_dataset = make_dataset(list(train_data.keys()), list(train_data.values()))

valid_dataset = make_dataset(list(valid_data.keys()), list(valid_data.values()))


In [ ]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1)

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:

def get_cnn_model():
    base_model = EfficientNetV2L(
        input_shape=(*IMAGE_SIZE, 3), include_top=False, weights="imagenet",
    )
    # We freeze our feature extractor
    base_model.trainable = False
    base_model_out = base_model.output
    base_model_out = layers.Reshape((-1, base_model_out.shape[-1]))(base_model_out)
    cnn_model = keras.models.Model(base_model.input, base_model_out)
    return cnn_model


class TransformerEncoderBlock(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.dense_1 = layers.Dense(dense_dim, activation="relu")
        self.dense_2 = layers.Dense(embed_dim)
        self.dropout = layers.Dropout(rate)

    def call(self, inputs, training, mask=None):

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=None,
            training=training,
        )

        out_1 = self.layernorm_1(inputs + attention_output_1)
        out_2 = self.dense_1(out_1)
        out_3 = self.dense_2(out_2)
        out_3 = self.dropout(out_3, training=training)

        return self.layernorm_2(out_1 + out_3)

class PositionalEmbedding_IMG(layers.Layer):
    def __init__(self, sequence_length, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.pos_encoding = positional_encoding(length=sequence_length, depth=embed_dim)
        self.sequence_length = sequence_length
        self.embed_dim = embed_dim

    def call(self, x):
        length = tf.shape(x)[1]
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

class Encoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, num_layers, rate=0.4, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_layers = num_layers
        self.embedding = layers.Dense(embed_dim, activation='relu')
        self.pos_embedding = PositionalEmbedding_IMG(
            embed_dim=embed_dim, sequence_length=r_size * c_size
        )
        self.enc_layers = [TransformerEncoderBlock(embed_dim, dense_dim, num_heads)
                       for _ in range(num_layers)]
        self.dropout = layers.Dropout(rate)
    def call(self, x, training, mask=None):
        x = self.embedding(x)
        x = self.pos_embedding(x)
        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
          x = self.enc_layers[i](x, training, mask)

        return x

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_encoding = positional_encoding(length=sequence_length, depth=embed_dim)
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, x):
        length = tf.shape(x)[1]
        x = self.token_embeddings(x)
        x *= tf.math.sqrt(tf.cast(self.embed_dim, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

class TransformerDecoderBlock(layers.Layer):
    def __init__(self, embed_dim, ff_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.ffn_layer_1 = layers.Dense(ff_dim, activation="relu")
        self.ffn_layer_2 = layers.Dense(embed_dim)

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

        self.dropout_1 = layers.Dropout(0.8)
        self.dropout_2 = layers.Dropout(0.8)
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, training, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is not None:
            padding_mask = tf.cast(mask[:, :, tf.newaxis], dtype=tf.int32)
            combined_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.int32)
            combined_mask = tf.minimum(combined_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=combined_mask,
            training=training,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2, attention_scores = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
            training=training,
            return_attention_scores=True
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        ffn_out = self.ffn_layer_1(out_2)
        ffn_out = self.ffn_layer_2(ffn_out)
        ffn_out = self.dropout_2(ffn_out, training=training)
        ffn_out = self.layernorm_3(ffn_out + out_2, training=training)

        return ffn_out, attention_scores

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

class Decoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, n_layers, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.n_layers = n_layers
        self.layernorm_1 = layers.LayerNormalization()
        self.out = layers.Dense(VOCAB_SIZE, activation="softmax")
        self.embedding = PositionalEmbedding(
            embed_dim=embed_dim, sequence_length=SEQ_LENGTH, vocab_size=VOCAB_SIZE
        )
        self.decoder_layers = [TransformerDecoderBlock(embed_dim=embed_dim, ff_dim=FF_DIM, num_heads=heads)
                                for _ in range(n_layers)]
  def call(self, inputs, encoder_outputs, training, mask=None):
        inputs = self.embedding(inputs)

        for i in range(self.n_layers):
            inputs, weights = self.decoder_layers[i](inputs, encoder_outputs, training, mask)
        pred = self.out(inputs)
        return pred
  def pred(self, inputs, encoder_outputs, training, mask=None):
        inputs = self.embedding(inputs)

        layers_att_weights = []
        for i in range(self.n_layers):
            inputs, weights = self.decoder_layers[i](inputs, encoder_outputs, training, mask)
            layers_att_weights.append(weights)
        pred = self.out(inputs)
        return pred, layers_att_weights

class ImageCaptioningModel(keras.Model):
    def __init__(
        self, cnn_model, encoder, decoder, num_captions_per_image=5, image_aug=None,
    ):
        super().__init__()
        self.cnn_model = cnn_model
        self.encoder = encoder
        self.decoder = decoder
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.acc_tracker = keras.metrics.Mean(name="accuracy")
        self.num_captions_per_image = num_captions_per_image
        self.image_aug = image_aug

    def calculate_loss(self, y_true, y_pred, mask):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

    def calculate_accuracy(self, y_true, y_pred, mask):
        accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
        accuracy = tf.math.logical_and(mask, accuracy)
        accuracy = tf.cast(accuracy, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)

    def _compute_caption_loss_and_acc(self, img_embed, batch_seq, training=True):
        encoder_out = self.encoder(img_embed, training=training)
        batch_seq_inp = batch_seq[:, :-1]
        batch_seq_true = batch_seq[:, 1:]
        mask = tf.math.not_equal(batch_seq_true, 0)
        batch_seq_pred = self.decoder(
            batch_seq_inp, encoder_out, training=training, mask=mask
        )
        loss = self.calculate_loss(batch_seq_true, batch_seq_pred, mask)
        acc = self.calculate_accuracy(batch_seq_true, batch_seq_pred, mask)
        return loss, acc

    def train_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        if self.image_aug:
            batch_img = self.image_aug(batch_img)

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            with tf.GradientTape() as tape:
                loss, acc = self._compute_caption_loss_and_acc(
                    img_embed, batch_seq[:, i, :], training=True
                )

                # 3. Update loss and accuracy
                batch_loss += loss
                batch_acc += acc

            # 4. Get the list of all the trainable weights
            train_vars = (
                self.encoder.trainable_variables + self.decoder.trainable_variables
            )

            # 5. Get the gradients
            grads = tape.gradient(loss, train_vars)

            # 6. Update the trainable weights
            self.optimizer.apply_gradients(zip(grads, train_vars))

        # 7. Update the trackers
        batch_acc /= float(self.num_captions_per_image)
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 8. Return the loss and accuracy values
        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    def test_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            loss, acc = self._compute_caption_loss_and_acc(
                img_embed, batch_seq[:, i, :], training=False
            )

            # 3. Update batch loss and batch accuracy
            batch_loss += loss
            batch_acc += acc

        batch_acc /= float(self.num_captions_per_image)

        # 4. Update the trackers
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 5. Return the loss and accuracy values
        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker, self.acc_tracker]


In [ ]:
# Dimension for the image embeddings and token embeddings
EMBED_DIM = 512
r_size, c_size=12, 12
# Per-layer units in the feed-forward network
FF_DIM = 2048

heads = 6
cnn_model = get_cnn_model()
encoder = Encoder(embed_dim=EMBED_DIM, dense_dim=FF_DIM, num_heads=heads, num_layers=2)
decoder = Decoder(embed_dim=EMBED_DIM, dense_dim=FF_DIM, num_heads=heads, n_layers=2)
caption_model = ImageCaptioningModel(
    cnn_model=cnn_model, encoder=encoder, decoder=decoder, image_aug=image_augmentation,
)
last_epoch=0

473186304/473176280 [==============================] - 20s 0us/step


## Model training

In [ ]:

# Define the loss function
cross_entropy = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction="none"
)

# EarlyStopping criteria
early_stopping = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6, verbose=1)
checkpoint_filepath = '/content/drive/MyDrive/Mestrado/ImageCaptioning/pesos/model_V2.{epoch:02d}-{val_loss:.4f}.tf'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# Learning Rate Scheduler for the optimizer
class LRSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, post_warmup_learning_rate, warmup_steps):
        super().__init__()
        self.post_warmup_learning_rate = post_warmup_learning_rate
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        global_step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        warmup_progress = global_step / warmup_steps
        warmup_learning_rate = self.post_warmup_learning_rate * warmup_progress
        return tf.cond(
            global_step < warmup_steps,
            lambda: warmup_learning_rate,
            lambda: self.post_warmup_learning_rate,
        )
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=10000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

EPOCHS=90
learning_rate = LRSchedule(post_warmup_learning_rate=1e-5, warmup_steps=20000)

# Compile the model
caption_model.compile(optimizer=keras.optimizers.Adam(learning_rate), loss=cross_entropy)

In [ ]:
# Fit the model
H=caption_model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=valid_dataset,
    callbacks=[early_stopping, model_checkpoint_callback],
    initial_epoch=last_epoch
)

Epoch 1/90
454/454 [==============================] - 311s 595ms/step - loss: 41.0697 - acc: 0.0351 - val_loss: 35.8497 - val_acc: 0.1083
Epoch 2/90
454/454 [==============================] - 258s 568ms/step - loss: 32.2299 - acc: 0.1199 - val_loss: 28.3558 - val_acc: 0.1708
Epoch 3/90
454/454 [==============================] - 256s 564ms/step - loss: 26.6515 - acc: 0.1816 - val_loss: 24.8406 - val_acc: 0.2079
Epoch 4/90
454/454 [==============================] - 258s 567ms/step - loss: 24.1565 - acc: 0.2134 - val_loss: 22.8970 - val_acc: 0.2363
Epoch 5/90
454/454 [==============================] - 257s 566ms/step - loss: 22.4438 - acc: 0.2427 - val_loss: 21.3489 - val_acc: 0.2660
Epoch 6/90
454/454 [==============================] - 258s 569ms/step - loss: 21.0966 - acc: 0.2681 - val_loss: 20.1673 - val_acc: 0.2867
Epoch 7/90
454/454 [==============================] - 256s 564ms/step - loss: 19.9672 - acc: 0.2904 - val_loss: 19.1909 - val_acc: 0.3028
Epoch 8/90
454/454 [==============